Created on Thu Apr 18 05:43:21 2024

@author: Santiago D'hers

Use:

- This script will create autolabels analyzing position files

Requirements:

- The position.csv files processed by 1-Manage_H5.py
- The desired model trained with 3a-Create_Models.py

In [18]:
import os
from glob import glob
import pandas as pd
import numpy as np

import tensorflow as tf

import joblib
from keras.models import load_model

In [ ]:
# State your path:
base = r'C:\Users\dhers\OneDrive - UBA\workshop'
experiment = r'Interferencia'
path = os.path.join(base, experiment)

all_position = glob(os.path.join(path,"T*/position/*position.csv"))

# Find the model you want to use
models_folder = 'C:/Users/dhers/Desktop/STORM/docs/models'
model_name = 'seq'
model_date = 'best' # 'best' or a specific date, e.g. '2024-11-13'

objects = ['obj_1', 'obj_2']
bodyparts = ['nose', 'L_ear', 'R_ear', 'head', 'neck', 'body']

rescale = True
reshape = True # True for LSTM models
if reshape:
    past, future = 3, 3 # if you use a LSTM model, you can set the window size here
    

In [ ]:
# Load the saved model from file

loaded_model = load_model(os.path.join(models_folder, f'{model_name}/model_{model_name}_{model_date}.keras'))
# loaded_model = joblib.load(os.path.join(models_folder, f'RF/model_RF_{use_model_date}.pkl'))


In [22]:
def recenter(df: pd.DataFrame, point: str, bodyparts: list) -> pd.DataFrame:
    """Recenters a DataFrame around a specified point.

    Args:
        df (pd.DataFrame): DataFrame to be recentered.
        point (str): Name of the point to be used as the center.
        bodyparts (list): List of bodyparts to be recentered.

    Returns:
        pd.DataFrame: Recentered DataFrame.
    """
    # Create a copy of the original dataframe
    df_copy = df.copy()
    bodypart_columns = []
    
    for bodypart in bodyparts:
        # Subtract point_x from columns ending in _x
        x_cols = [col for col in df_copy.columns if col.endswith(f'{bodypart}_x')]
        df_copy[x_cols] = df_copy[x_cols].apply(lambda col: col - df_copy[f'{point}_x'])
        
        # Subtract point_y from columns ending in _y
        y_cols = [col for col in df_copy.columns if col.endswith(f'{bodypart}_y')]
        df_copy[y_cols] = df_copy[y_cols].apply(lambda col: col - df_copy[f'{point}_y'])
        
        # Collect bodypart columns
        bodypart_columns.extend(x_cols)
        bodypart_columns.extend(y_cols)
        
    return df_copy[bodypart_columns]

In [23]:
def reshape(df: pd.DataFrame, past: int = 3, future: int = 3, broaden: float = 1.7) -> np.ndarray:
    """Reshapes a DataFrame into a 3D NumPy array.

    Args:
        df (pd.DataFrame): DataFrame to reshape.
        past (int, optional): Number of past frames to include. Defaults to 3.
        future (int, optional): Number of future frames to include. Defaults to 3.
        broaden (float, optional): Factor to broaden the range of frames. Defaults to 1.7.

    Returns:
        np.ndarray: 3D NumPy array with shape (past + future + 1, past + future + 1, 2).
    """

    reshaped_df = []
    
    frames = list(range(-past, future + 1))

    if broaden > 1:
        frames = [-int(abs(x) ** broaden) if x < 0 else int(x ** broaden) for x in frames]

    # Iterate over each row index in the DataFrame
    for i in range(len(df)):
        # Determine which indices to include for reshaping
        indices_to_include = sorted([
            max(0, i - frame) if frame > 0 else min(len(df) - 1, i - frame)
            for frame in frames
        ])
        
        # Append the rows using the calculated indices
        reshaped_df.append(df.iloc[indices_to_include].to_numpy())
    
    # Convert the list to a 3D NumPy array
    reshaped_array = np.array(reshaped_df)
    
    return reshaped_array

In [26]:
def use_model(position: pd.DataFrame, model: object, objects: list = ['obj_1', 'obj_2'], bodyparts: list = ['nose', 'L_ear', 'R_ear', 'head', 'neck', 'body'], recentering: bool = True, reshaping: bool = False) -> pd.DataFrame:
    """Use the model to predict labels for each object in the DataFrame.

    Args:
        position (pd.DataFrame): DataFrame containing the position data.
        model (object): Model object to use for prediction.
        objects (list, optional): List of objects to predict labels for. Defaults to ['obj_1', 'obj_2'].
        bodyparts (list, optional): List of bodyparts to include in the model. Defaults to ['nose', 'L_ear', 'R_ear', 'head', 'neck', 'body'].
        recentering (bool, optional): Whether to recenter the DataFrame. Defaults to True.
        reshaping (bool, optional): Whether to reshape the DataFrame. Defaults to False.

    Returns:
        pd.DataFrame: DataFrame containing the predicted labels.
    """
    
    if recentering:
        dfs = []
        for obj in objects:
            recentered = recenter(position, obj, bodyparts)
            dfs.append(recentered)
        position = pd.concat(dfs,ignore_index=True)
    
    if reshaping:
        position = np.array(reshape(position))
    
    pred = model.predict(position) # Use the model to predict the labels
    pred = pred.flatten()
    pred = pd.DataFrame(pred, columns=['predictions'])

    n_objects = len(objects)

    # Calculate the length of each fragment
    fragment_length = len(pred) // n_objects

    # Create a list to hold each fragment
    fragments = [pred.iloc[i*fragment_length:(i+1)*fragment_length].reset_index(drop=True) for i in range(n_objects)]

    # Concatenate fragments along columns
    labels = pd.concat(fragments, axis=1)

    # Rename columns
    labels.columns = [f'{obj}' for obj in objects]

    labels = round(labels, 2)
    
    return labels

In [27]:
def create_autolabels(files: list, model: object, objects: list = ['obj_1', 'obj_2'], bodyparts: list = ['nose', 'L_ear', 'R_ear', 'head', 'neck', 'body'], rescaling: bool = True, reshaping: bool = False):
    """Creates autolabels for a list of files.

    Args:
        files (list): List of files to create autolabels for.
        model (object): Model object to use for autolabeling.
        objects (list, optional): List of objects to use for autolabeling. Defaults to ['obj_1', 'obj_2'].
        bodyparts (list, optional): List of bodyparts to use for autolabeling. Defaults to ['nose', 'L_ear', 'R_ear', 'head', 'neck', 'body'].
        rescaling (bool, optional): Whether to rescale the data. Defaults to True.
        reshaping (bool, optional): Whether to reshape the data. Defaults to False.
    """
    
    for file in files:
        
        # Determine the output file path
        input_dir, input_filename = os.path.split(file)
        parent_dir = os.path.dirname(input_dir)
        
        # Read the file
        position = pd.read_csv(file)
    
        # lets analyze it!
        autolabels = use_model(position, model, objects, bodyparts, rescaling, reshaping)
        
        # Set column names and add a new column "Frame" with row numbers
        autolabels.insert(0, "Frame", autolabels.index + 1)
    
        # Create a filename for the output CSV file
        output_filename = input_filename.replace('_position.csv', '_autolabels.csv')
        output_folder = os.path.join(parent_dir + '/autolabels')
        os.makedirs(output_folder, exist_ok = True)
        output_path = os.path.join(output_folder, output_filename)
        
        # Save autolabels to a CSV file
        autolabels.to_csv(output_path, index=False)
        print(f"Saved autolabels to {output_filename}")

In [28]:
create_autolabels(all_position, loaded_model, objects, bodyparts, rescaling = rescale, reshaping = reshape) # Lets analyze!

457/457 [==============================] - 7s 10ms/step
Saved autolabels to 2023-11_Interferencia_TR1_R01_C01_A_L_autolabels.csv
460/460 [==============================] - 5s 11ms/step
Saved autolabels to 2023-11_Interferencia_TR1_R02_C01_A_R_autolabels.csv
461/461 [==============================] - 5s 11ms/step
Saved autolabels to 2023-11_Interferencia_TR1_R03_C01_B_L_autolabels.csv
466/466 [==============================] - 5s 10ms/step
Saved autolabels to 2023-11_Interferencia_TR1_R04_C01_B_R_autolabels.csv
464/464 [==============================] - 5s 11ms/step
Saved autolabels to 2023-11_Interferencia_TR1_R05_C02_A_L_autolabels.csv
466/466 [==============================] - 5s 10ms/step
Saved autolabels to 2023-11_Interferencia_TR1_R06_C02_A_R_autolabels.csv
463/463 [==============================] - 5s 10ms/step
Saved autolabels to 2023-11_Interferencia_TR1_R07_C02_B_L_autolabels.csv
462/462 [==============================] - 5s 10ms/step
Saved autolabels to 2023-11_Interferencia